In [13]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [14]:
PINECONE_API_KEY = "6b88f8d7-b7e8-4ec3-af39-eb1f92a9f91d"
PINECONE_API_ENV = "gcp-starter"

In [15]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [16]:
extracted_data = load_pdf(r"C:\Users\mouni\Desktop\projects\diet_guidelines_chatbot\data")

In [17]:
extracted_data

[Document(page_content='', metadata={'source': 'C:\\Users\\mouni\\Desktop\\projects\\diet_guidelines_chatbot\\data\\DietaryGuidelinesforNINwebsite.pdf', 'page': 0}),
 Document(page_content='DIETARY GUIDELINES \nFOR INDIANS \n-A Manual \nNATIONAL INSTITUTE OF NUTRITION \nHyderabad – 500 007, INDIA ', metadata={'source': 'C:\\Users\\mouni\\Desktop\\projects\\diet_guidelines_chatbot\\data\\DietaryGuidelinesforNINwebsite.pdf', 'page': 1}),
 Document(page_content='First Published                 .....           1998 \nReprinted                         .....           1999, 2003, 2005, 2007 Second Edition                .....            2011 \nATTENTION  READERS \nReaders can download the soft copy of the “Dietary Guidelines \nfor Indians” free of cost for their own use and also for \ndissemination of information for  the benefit of the society on non- \nprofit basis. Readers are not supposed to either reproduce / copy / \nreprint the same for commercial purpose. If the information is used \

In [18]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [19]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 511


In [20]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [21]:
embeddings = download_hugging_face_embeddings()

In [22]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [23]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="food-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [15]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is a balanced diet?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='A balanced diet is one which provides all the nutrients in required amounts and \nproper proportions. It can easily be achieved through a blend of the four basic food \ngroups.  The quantities of foods needed to meet the nutrient requirements vary with \nage, gender, physiological status and physical activity. A balanced diet should \nprovide around 50-60% of total calories from carbohydrates, preferably from \ncomplex carbohydrates, about 10-15% from proteins and 20-30% from both visible', metadata={}), Document(page_content='Balanced Diet for Adults - Sedentary/Moderate/Heavy Activity 86', metadata={}), Document(page_content='Guideline 1 \nEat variety of foods to ensure a balanced diet \nRationale: Nutritionally adequate diet should be consumed through a \nwise choice from a variety of foods \n/c118Nutrition is a basic prerequisite to sustain life. \n/c118Variety in food is not only the spice of life but also the essence of nutrition and \nhealth. \n/c1

In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [28]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [30]:
llm=CTransformers(model=r"C:\Users\mouni\Desktop\FOOD NUTRITION CHATBOT\foodnutrition_chatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [31]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [32]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  
Yes, it is possible to use a 93-key keyboard with a computer. There are different types of keyboard layouts that can be used with a 93-key keyboard, such as QWERTY or Dvorak. The most common type of keyboard layout used with a 93-key keyboard is the Dvorak layout, which was designed to reduce finger movement and increase typing speed.


PineconeProtocolError: Failed to connect; did you specify the correct index name?